# **Deep Reinforcement Learning Class Spring 2025 Assignment 1**

In this assignment, we will learn about gym interface, gridworld, q-learning, and etc. You will need to fill in the missing code snippets (marked by TODO).

Make a copy of this notebook using File > Save a copy in Drive and edit it with your answers.

WARNING: Do not put your name or any other personal identification information in this notebook.

In [3]:
!pip install gym
!pip install numpy==1.24.0
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\yuhun\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.24.0.tar.gz (10.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\yuhun\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\yuhun\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pip\_internal\cli\base_command.py", line 105, in _run_wrapper
    status = _inner_run()
             ^^^^^^^^^^^^
  File "C:\Users\yuhun\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pip\_internal\cli\base_command.py", line 96, in _inner_run
    return self.run(options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\yuhun\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pip\_internal\cli\req_command.py",

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\yuhun\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import gym
import numpy as np
import importlib.util
import time
from IPython.display import clear_output
import random

In [5]:
np.random.seed(42)

In [6]:

# This environment allows you to verify whether your program runs correctly during testing,
# as it follows the same observation format from `env.reset()` and `env.step()`.
# However, keep in mind that this is just a simplified environment.
# The full specifications for the real testing environment can be found in the provided spec.
#
# You are free to modify this file to better match the real environment and train your own agent.
# Good luck!


class SimpleTaxiEnv(gym.Env):
    def __init__(self, grid_size=10, fuel_limit=5000):
        """
        Custom Taxi environment supporting different grid sizes.
        """
        self.grid_size = grid_size
        self.fuel_limit = fuel_limit
        self.current_fuel = fuel_limit
        self.passenger_picked_up = False

        self.stations = [(0, 0), (0, self.grid_size - 1), (self.grid_size - 1, 0), (self.grid_size - 1, self.grid_size - 1)]
        self.passenger_loc = None

        self.obstacles = set()  # No obstacles in simple version
        self.destination = None
    def legal(self):
      available_positions = [
            (x, y) for x in range(self.grid_size) for y in range(self.grid_size)
            if (x, y) not in self.obstacles
      ]
      n=len(available_positions)
      p = [0 for i in range(n)]
      global group_size
      group_size = n
      def find_parent(x):
        if x!=p[x]:
          p[x]=find_parent(p[x])
        return p[x]
      def Union(a,b):
        global group_size
        a=find_parent(a)
        b=find_parent(b)
        if a!=b:
          group_size -= 1
          if np.random.randint(2)==0:
            p[a]=b
          else:
            p[b]=a
      for i in range(n):
        p[i]=i
      for i in range(n):
        for j in range(i+1,n):
          if abs(available_positions[i][0]-available_positions[j][0])+abs(available_positions[i][1]-available_positions[j][1])==1:
            Union(i,j)
      return group_size==1
    def reset(self,grid_size=None,obstacle_len=None):
        """Reset the environment, ensuring Taxi, passenger, and destination are not overlapping obstacles"""
        self.current_fuel = self.fuel_limit
        self.passenger_picked_up = False
        if grid_size == None:
            grid_size = np.random.randint(5,11)
        self.grid_size = grid_size
        self.stations = []
        if obstacle_len == None:
            obstacle_len = np.random.randint(0,grid_size*grid_size)
        while len(self.stations)<4:
            posx,posy = np.random.randint(0,self.grid_size),np.random.randint(0,self.grid_size)
            # check posx,posy is not adjacent to existed stations
            if not any([abs(posx-station[0])+abs(posy-station[1])<=1 for station in self.stations]):
                self.stations.append((posx,posy))
        #self.stations = [(0, 0), (0, self.grid_size - 1), (self.grid_size - 1, 0), (self.grid_size - 1, self.grid_size - 1)]
        self.obstacles = set()
        cnt = 0
        while len(self.obstacles)<obstacle_len and cnt < self.grid_size*self.grid_size*10:
            posx,posy = np.random.randint(0,self.grid_size),np.random.randint(0,self.grid_size)
            if (posx,posy) not in self.stations and (posx,posy) not in self.obstacles:
                self.obstacles.add((posx,posy))
            if not self.legal():
                self.obstacles.remove((posx,posy))
            cnt = cnt +1
        available_positions = [
            (x, y) for x in range(self.grid_size) for y in range(self.grid_size)
            if (x, y) not in self.stations and (x, y) not in self.obstacles
        ]

        self.taxi_pos = random.choice(available_positions)

        self.passenger_loc = random.choice([pos for pos in self.stations])


        possible_destinations = [s for s in self.stations if s != self.passenger_loc]
        self.destination = random.choice(possible_destinations)
        #print(f'grid size : {self.grid_size}, obstacles : {len(self.obstacles)}, ')
        #self.render_env(self.taxi_pos)
        return self.get_state(), {}

    def step(self, action):
        """Perform an action and update the environment state."""
        taxi_row, taxi_col = self.taxi_pos
        next_row, next_col = taxi_row, taxi_col
        reward = 0
        if action == 0 :  # Move Down
            next_row += 1
        elif action == 1:  # Move Up
            next_row -= 1
        elif action == 2:  # Move Right
            next_col += 1
        elif action == 3:  # Move Left
            next_col -= 1


        if action in [0, 1, 2, 3]:  # Only movement actions should be checked
            if (next_row, next_col) in self.obstacles or not (0 <= next_row < self.grid_size and 0 <= next_col < self.grid_size):
                reward -=10
                #print('block')
            else:
                self.taxi_pos = (next_row, next_col)
                if self.passenger_picked_up:
                    self.passenger_loc = self.taxi_pos
        else:
            if action == 4:  # PICKUP
                if self.taxi_pos == self.passenger_loc:
                    self.passenger_picked_up = True
                    self.passenger_loc = self.taxi_pos
                else:
                    reward = -10
            elif action == 5:  # DROPOFF
                if self.passenger_picked_up:
                    self.passenger_picked_up = False
                    self.passenger_loc = self.taxi_pos
                    if self.taxi_pos == self.destination:
                        reward += 50
                        return self.get_state(), reward -0.1, True, {}
                    else:
                        reward -=10
                else:
                    reward -=10

        reward -= 0.1

        self.current_fuel -= 1
        if self.current_fuel <= 0:
            return self.get_state(), reward -10, True, {}



        return self.get_state(), reward, False, {}

    def get_state(self):
        """Return the current environment state."""
        taxi_row, taxi_col = self.taxi_pos
        passenger_row, passenger_col = self.passenger_loc
        destination_row, destination_col = self.destination

        obstacle_north = int(taxi_row == 0 or (taxi_row-1, taxi_col) in self.obstacles)
        obstacle_south = int(taxi_row == self.grid_size - 1 or (taxi_row+1, taxi_col) in self.obstacles)
        obstacle_east  = int(taxi_col == self.grid_size - 1 or (taxi_row, taxi_col+1) in self.obstacles)
        obstacle_west  = int(taxi_col == 0 or (taxi_row , taxi_col-1) in self.obstacles)

        passenger_loc_north = int((taxi_row - 1, taxi_col) == self.passenger_loc)
        passenger_loc_south = int((taxi_row + 1, taxi_col) == self.passenger_loc)
        passenger_loc_east  = int((taxi_row, taxi_col + 1) == self.passenger_loc)
        passenger_loc_west  = int((taxi_row, taxi_col - 1) == self.passenger_loc)
        passenger_loc_middle  = int( (taxi_row, taxi_col) == self.passenger_loc)
        passenger_look = passenger_loc_north or passenger_loc_south or passenger_loc_east or passenger_loc_west or passenger_loc_middle

        destination_loc_north = int( (taxi_row - 1, taxi_col) == self.destination)
        destination_loc_south = int( (taxi_row + 1, taxi_col) == self.destination)
        destination_loc_east  = int( (taxi_row, taxi_col + 1) == self.destination)
        destination_loc_west  = int( (taxi_row, taxi_col - 1) == self.destination)
        destination_loc_middle  = int( (taxi_row, taxi_col) == self.destination)
        destination_look = destination_loc_north or destination_loc_south or destination_loc_east or destination_loc_west or destination_loc_middle


        state = (taxi_row, taxi_col, self.stations[0][0],self.stations[0][1] ,self.stations[1][0],self.stations[1][1],self.stations[2][0],self.stations[2][1],self.stations[3][0],self.stations[3][1],obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look)
        return state,self.passenger_picked_up,self.passenger_loc
    def render_env(self, taxi_pos,   action=None, step=None, fuel=None):
        clear_output(wait=True)

        grid = [['.'] * self.grid_size for _ in range(self.grid_size)]

        '''
        # Place passenger
        py, px = passenger_pos
        if 0 <= px < self.grid_size and 0 <= py < self.grid_size:
            grid[py][px] = 'P'
        '''
        stations_char =['R','G','B','Y']
        for i in range(len(self.stations)):
            sx,sy = self.stations[i]
            grid[sy][sx] = stations_char[i]
        for ox,oy in self.obstacles:
            grid[oy][ox] = '#'

        '''
        # Place destination
        dy, dx = destination_pos
        if 0 <= dx < self.grid_size and 0 <= dy < self.grid_size:
            grid[dy][dx] = 'D'
        '''
        # Place taxi
        ty, tx = taxi_pos
        if 0 <= tx < self.grid_size and 0 <= ty < self.grid_size:
            grid[ty][tx] = '🚖'

        # Print step info
        print(f"\nStep: {step}")
        print(f"Taxi Position: ({tx}, {ty})")
        #print(f"Passenger Position: ({px}, {py}) {'(In Taxi)' if (px, py) == (tx, ty) else ''}")
        #print(f"Destination: ({dx}, {dy})")
        print(f"Fuel Left: {fuel}")
        print(f"Last Action: {self.get_action_name(action)}\n")

        # Print grid
        for row in grid:
            print(" ".join(row))
        print("\n")

    def get_action_name(self, action):
        """Returns a human-readable action name."""
        actions = ["Move South", "Move North", "Move East", "Move West", "Pick Up", "Drop Off"]
        return actions[action] if action is not None else "None"

## 狀態定義

## 狀態定義

- pos (相對 R/G/Y/B)的距離 (用 +-0)去代表，直到走到一位置確認是否 pickup/destination為止後 random 換位置。
    - 只要相對位移>0一律用 +1表示, <0 用 -1
- obstacles
- passenger state
    - 0 ⇒ not on car
    - 1 ⇒ on car
- candidates_p
    - original : stations
    - whenever one is not worked ⇒ removed it
- mask_p= [1] * len(candidates_p) + [0] * (4-len(candidates_p))
    - 0 代表 passenger_i 是不可能的位置
    - 1 代表可能
- candidates_goal
    - original : stations
    - whenever one is not goal ⇒ removed it
- mask_goal= [1] * len(candidates_goal) + [0] * (4-len(candidates_goal))
    - 0 代表 passenger_i 是不可能的位置
    - 1 代表可能
- pickup
- goal_id : {0,1,2,3}
    - 目標是 stations[goal_id]


### 轉移

- relative_pos = (0,0)
    - if passenger_look
        - candidate_p = [pos]
        - success pickup
            - success pickup ⇒ pickup_id=True
            - candidates_p = [pos]
            - goal = random choice (candidates_goal)
    - else:
        - candidate_p remove pos
        - goal = random.choice(candidates_p)
    - if goal_look
        - candidate_g = [pos]
    - else:
        - candidate_g.remove(pos)
        - goal = random choice (candidates_goal)
- drop
    - if pickup ⇒ candidates_p = [pos], goal_id = 0

In [26]:
from re import M
from collections import defaultdict
from tqdm import tqdm
def tabular_q_learning(env,episodes=20000, alpha=0.1, gamma=0.99,
                       epsilon_start=1.0, epsilon_end=0.1, decay_rate=0.9997):

    global stations, candidates_p,candidates_goal, pickup
    stations = [[0,0] for _ in range(4)]
    candidates_p = [i for i in stations]
    candidates_goal = [i for i in stations]
    pickup=False
    action_size = 6
    q_table = defaultdict(lambda :np.zeros(action_size))
    pickup_id = 4
    drop_id = 5
    ifpickup=False
    p_loc = (0,0)
    def cmp(a,b):
        if a==b:
            return 0
        return 1 if a<b else -1
    def get_state_obs(obs,action):
        global stations,pickup,candidates_p,candidates_goal
        #print(candidates_p)
        taxi_row, taxi_col, stations[0][0], stations[0][1] , stations[1][0], stations[1][1],stations[2][0],stations[2][1],stations[3][0],stations[3][1],obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs
        agent_pos = (taxi_row,taxi_col)
        if action==None:
            # initialize
            candidates_goal = [tuple(i) for i in stations]
            candidates_p = [tuple(i) for i in stations]
            pickup=False
        if passenger_look:
            #print('before p',candidates_p)
            candidates_p = [ tuple(x) for x in candidates_p if abs(x[0]-agent_pos[0])+abs(x[1]-agent_pos[1]) <=1 ]
            #print('after p',candidates_p)
        else:
            #print('before p',candidates_p)
            candidates_p = [ tuple(x) for x in candidates_p if x != agent_pos ]
            #print('after p',candidates_p)
        if destination_look:
            #print('before g',candidates_goal)
            candidates_goal = [ tuple(x) for x in candidates_goal if abs(x[0]-agent_pos[0])+abs(x[1]-agent_pos[1]) <=1 ]
            #print('after g',candidates_goal)
        else:
            #print('before g',candidates_goal)
            candidates_goal = [ tuple(x) for x in candidates_goal if x!=agent_pos ]
            #print('after g',candidates_goal)
        if action==pickup_id and not pickup and agent_pos in candidates_p:
            pickup = True
            candidates_p = []
        elif action == drop_id and pickup:
            pickup=False
            candidates_p.append(agent_pos)
        cmp_pos = (0,0)
        if not pickup:
            cmp_pos = candidates_p[0]
        else:
            cmp_pos = candidates_goal[0]
        passenger_look = passenger_look and agent_pos in candidates_p
        destination_look = destination_look and agent_pos in candidates_goal
        relative_pos = (cmp(agent_pos[0],cmp_pos[0]),cmp(agent_pos[1],cmp_pos[1]))
        return (relative_pos,pickup, passenger_look, destination_look, (obstacle_north,obstacle_south,obstacle_east,obstacle_west))
    """
    if action == 0 :  # Move Down
        next_row += 1
    elif action == 1:  # Move Up
        next_row -= 1
    elif action == 2:  # Move Right
        next_col += 1
    elif action == 3:  # Move Left
        next_col -= 1
    """
    station_size = 4
    total_reward = 0
    total_reward_shaped = 0
    cnt = [0,0,0,0]
    epsilon = epsilon_start
    averaged = [0,0]
    batch_size = 100
    for epoch in tqdm(range(episodes+100)):
        if epoch >=episodes:
            epsilon = 0
        grid_size = 5 #np.random.randint(5,11)
        obstacle_size = np.random.randint(grid_size*grid_size)
        obs,_ = env.reset(grid_size,obstacle_size)
        averaged[0]+=env.grid_size
        averaged[1]+=len(env.obstacles)
        obs,ifpickup,p_loc = obs
        done = False
        state = get_state_obs(obs,action=None)
        steps=0
        action_l=[]
        success = False
        has_pickup=False
        while not done:
            """if state[-1]!=(0,0,0,0):
              print(state[-1])"""
            if np.random.choice(2,p=[epsilon,1-epsilon])==0 or state not in q_table.keys():
                action = np.random.randint(6)
            else:
                action = np.argmax(q_table[state])
            lst_pickup = pickup
            relative_pos,pickup, _,_ , _ = state
            obs,reward,done,_ = env.step(action)
            obs,ifpickup,p_loc = obs
            next_state = get_state_obs(obs,action)
            total_reward += reward

            ### reward shaping
            reward_shaping = 0
            """if relative_pos == (0,0):
                # want it to go to possible goal
                #reward_shaping += 0.5"""
            if relative_pos != (0,0) and action in [pickup_id,drop_id]:
                reward_shaping -= 10
            if done and reward>0:
                cnt[2]+=1
            
            relative_pos,pickup, _,_ , _ = next_state
            if not done and reward <-10 and action in [0,1,2,3]:
                #print('hit wall')
                cnt[-1]+=1
            """if not pickup:
                reward_shaping -= p_len
            reward_shaping -= 0.1 * goal_len"""
            reward += reward_shaping
            total_reward_shaped += reward
            if epsilon:
                q_table[state][action] = q_table[state][action] + alpha*(reward+gamma*np.max(q_table[next_state])-q_table[state][action])
            state = next_state
            if lst_pickup==False and pickup:
                cnt[0]+=1
            elif lst_pickup==True and pickup==False:
                cnt[1]+=1
            if pickup!=ifpickup:
                print(pickup,ifpickup)
            assert(pickup==ifpickup)


        if (epoch+1)%batch_size==0:
            cnt = [i/batch_size for i in cnt]
            print(f'Epsilon : {epsilon}, average reward : {total_reward/batch_size:.4f}, averaged shaped reward : {total_reward_shaped/batch_size:.4f} Pickup, Drop, Success, Hit wall rate : {cnt}')
            print(f'averaged grid size : {averaged[0]/batch_size:.2f}, averaged obstacles : {averaged[1]/batch_size:.2f}')
            averaged = [0,0]
            cnt = [0,0,0,0]
            total_reward = 0
            total_reward_shaped = 0
        epsilon *= decay_rate #max(epsilon*decay_rate ,epsilon_end)
    return q_table



In [27]:
env = SimpleTaxiEnv()
q_table = tabular_q_learning(env)

  1%|          | 102/20100 [00:06<17:31, 19.03it/s]

Epsilon : 0.9707323853954611, average reward : -12078.2520, averaged shaped reward : -18317.0520 Pickup, Drop, Success, Hit wall rate : [24.5, 24.48, 0.82, 569.13]
averaged grid size : 5.00, averaged obstacles : 9.55


  1%|          | 203/20100 [00:11<12:56, 25.62it/s]

Epsilon : 0.9420386676463453, average reward : -9305.8620, averaged shaped reward : -14093.7620 Pickup, Drop, Success, Hit wall rate : [23.93, 23.93, 0.91, 440.53]
averaged grid size : 5.00, averaged obstacles : 10.23


  2%|▏         | 303/20100 [00:15<08:53, 37.09it/s]

...

Epsilon : 0.002552720478632909, average reward : -355.4750, averaged shaped reward : -384.1750 Pickup, Drop, Success, Hit wall rate : [0.98, 0.81, 0.42, 2.7]
averaged grid size : 5.00, averaged obstacles : 10.69


100%|█████████▉| 20001/20100 [13:20<00:06, 16.13it/s]

Epsilon : 0.002477265036939326, average reward : -265.9780, averaged shaped reward : -288.4780 Pickup, Drop, Success, Hit wall rate : [1.05, 0.93, 0.6, 1.82]
averaged grid size : 5.00, averaged obstacles : 9.91


100%|██████████| 20100/20100 [13:26<00:00, 24.92it/s]

Epsilon : 0, average reward : -803.0920, averaged shaped reward : -803.0920 Pickup, Drop, Success, Hit wall rate : [0.54, 0.37, 0.37, 49.98]
averaged grid size : 5.00, averaged obstacles : 10.67


In [28]:
import pickle
q_table_dict = dict(q_table)  # Convert to regular dict
with open('q_table.pkl', 'wb') as f:
    pickle.dump(q_table_dict, f)

In [57]:
print(len(q_table_dict.keys()))

405


# 測試

In [ ]:
from tqdm import tqdm
def Testing(env):
    import pickle
    from collections import defaultdict
    with open('q_table.pkl', 'rb') as f:
        print('load')
        loaded_dict = pickle.load(f)
    q_table = defaultdict(lambda: np.zeros(6), loaded_dict)  # Replace 0 with your default value
    print(len(q_table))
    #print('len of q_table',len(q_table.keys()))
    global stations, candidates_p,candidates_goal, pickup, goal_id,last_action
    stations = [[0,0] for _ in range(4)]
    candidates_p = [i for i in stations]
    candidates_goal = [i for i in stations]
    goal_id = -1
    pickup=False
    action_size = 6
    last_action = None
    #q_table = defaultdict(lambda :np.zeros(action_size))
    pickup_id = 4
    drop_id = 5
    ifpickup=False
    p_loc = (0,0)
    def cmp(a,b):
        if a==b:
            return 0
        return 1 if a<b else -1
            
    def get_state_obs(obs,action):
        global stations,pickup,candidates_p,candidates_goal
        #print(candidates_p)
        taxi_row, taxi_col, stations[0][0], stations[0][1] , stations[1][0], stations[1][1],stations[2][0],stations[2][1],stations[3][0],stations[3][1],obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs
        agent_pos = (taxi_row,taxi_col)
        if action==None:
            # initialize
            candidates_goal = [tuple(i) for i in stations]
            candidates_p = [tuple(i) for i in stations]
            pickup=False
        if passenger_look:
            #print('before p',candidates_p)
            candidates_p = [ tuple(x) for x in candidates_p if abs(x[0]-agent_pos[0])+abs(x[1]-agent_pos[1]) <=1 ]
            #print('after p',candidates_p)
        else:
            #print('before p',candidates_p)
            candidates_p = [ tuple(x) for x in candidates_p if x != agent_pos ]
            #print('after p',candidates_p)
        if destination_look:
            #print('before g',candidates_goal)
            candidates_goal = [ tuple(x) for x in candidates_goal if abs(x[0]-agent_pos[0])+abs(x[1]-agent_pos[1]) <=1 ]
            #print('after g',candidates_goal)
        else:
            #print('before g',candidates_goal)
            candidates_goal = [ tuple(x) for x in candidates_goal if x!=agent_pos ]
            #print('after g',candidates_goal)
        if action==pickup_id and not pickup and agent_pos in candidates_p:
            pickup = True
            candidates_p = []
        elif action == drop_id and pickup:
            pickup=False
            candidates_p.append(agent_pos)
        cmp_pos = (0,0)
        if not pickup:
            cmp_pos = candidates_p[0]
        else:
            cmp_pos = candidates_goal[0]
        passenger_look = passenger_look and agent_pos in candidates_p
        destination_look = destination_look and agent_pos in candidates_goal
        relative_pos = (cmp(agent_pos[0],cmp_pos[0]),cmp(agent_pos[1],cmp_pos[1]))
        return (relative_pos,pickup, passenger_look, destination_look, (obstacle_north,obstacle_south,obstacle_east,obstacle_west))

    def get_action(obs):
        # TODO: Train your own agent
        # HINT: If you're using a Q-table, consider designing a custom key based on `obs` to store useful information.
        # NOTE: Keep in mind that your Q-table may not cover all possible states in the testing environment.
        #       To prevent crashes, implement a fallback strategy for missing keys.
        #       Otherwise, even if your agent performs well in training, it may fail during testing.
        global last_action
        state = get_state_obs(obs,last_action)
        action_name = ['Move North','Move South','Move East','Move West','Pick Up','Drop Off']
        if state not in q_table.keys():
            #print(state)
            print(state)
            assert(0)
            action = np.random.randint(action_size)
        else:
            #print(state,action_name[np.argmax(q_table[state])])
            action = np.argmax(q_table[state])
        q_table[state][action] = q_table[state][action] + 0.089487*(-0.1+0.89487*np.max(q_table[state])-q_table[state][action])
        last_action=action
        return action # Choose a random action
        # You can submit this random agent to evaluate the performance of a purely random strategy.
    Total_reward=0
    for i in tqdm(range(100)):
        grid_size = np.random.randint(5,11)
        obstacle_size = np.random.randint(grid_size*grid_size)
        obs,_ = env.reset()
        obs,_,_ = obs
        total_reward = 0
        done = False
        stations = [[0,0] for _ in range(4)]
        candidates_p = [i for i in stations]
        candidates_goal = [i for i in stations]
        goal_id = -1
        pickup=False
        action_size = 6
        pickup_id = 4
        drop_id = 5
        last_action = None
        lst_a = None
        while not done:
            assert(last_action==lst_a)
            action = get_action(obs)
            obs,reward,done,_ = env.step(action)
            obs,_,_ = obs
            total_reward += reward
            lst_a=action
        Total_reward+=total_reward
        print(f'grid_size : {env.grid_size}, obstacle_size : {len(env.obstacles)}, total_reward : {total_reward}')
    print(f'average : {Total_reward/100}')


In [30]:
env = SimpleTaxiEnv()
Testing(env)

FileNotFoundError: [Errno 2] No such file or directory: 'q_table copy.pkl'

# Policy base

In [44]:
def tabular_policy_learning(env,episodes=5000, alpha=0.1, gamma=0.999):

    global stations, candidates_p,candidates_goal, pickup
    stations = [[0,0] for _ in range(4)]
    candidates_p = [i for i in stations]
    candidates_goal = [i for i in stations]
    pickup=False
    action_size = 6
    policy_table = defaultdict(lambda :np.zeros(action_size))
    pickup_id = 4
    drop_id = 5
    ifpickup=False
    p_loc = (0,0)
    def softmax(x):
        """✅ Compute softmax values for an array."""
        exp_x = np.exp(x - np.max(x))  # Numeric stability
        return exp_x / exp_x.sum()
    def get_state_obs(obs,action):
        global stations,pickup,candidates_p,candidates_goal
        #print(candidates_p)
        obstacles = [0 for _ in range(5)]
        taxi_row, taxi_col, stations[0][0], stations[0][1] , stations[1][0], stations[1][1],stations[2][0],stations[2][1],stations[3][0],stations[3][1],obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs
        agent_pos = (taxi_row,taxi_col)
        if action==None:
            # initialize
            candidates_goal = [tuple(i) for i in stations]
            candidates_p = [tuple(i) for i in stations]
            pickup=False
        if passenger_look:
            #print('before p',candidates_p)
            candidates_p = [ tuple(x) for x in candidates_p if abs(x[0]-agent_pos[0])+abs(x[1]-agent_pos[1]) <=1 ]
            #print('after p',candidates_p)
        else:
            #print('before p',candidates_p)
            candidates_p = [ tuple(x) for x in candidates_p if abs(x[0]-agent_pos[0])+abs(x[1]-agent_pos[1]) >1 ]
            #print('after p',candidates_p)
        if destination_look:
            #print('before g',candidates_goal)
            candidates_goal = [ tuple(x) for x in candidates_goal if abs(x[0]-agent_pos[0])+abs(x[1]-agent_pos[1]) <=1 ]
            #print('after g',candidates_goal)
        else:
            #print('before g',candidates_goal)
            candidates_goal = [ tuple(x) for x in candidates_goal if abs(x[0]-agent_pos[0])+abs(x[1]-agent_pos[1]) >1 ]
            #print('after g',candidates_goal)
        if action==pickup_id and not pickup and agent_pos in candidates_p:
            pickup = True
            candidates_p = []
        elif action == drop_id and pickup:
            pickup=False
            candidates_p = [agent_pos]
        return (pickup, len(candidates_p), len(candidates_goal), passenger_look, destination_look, (obstacle_north,obstacle_south,obstacle_east,obstacle_west))
    """
    if action == 0 :  # Move Down
        next_row += 1
    elif action == 1:  # Move Up
        next_row -= 1
    elif action == 2:  # Move Right
        next_col += 1
    elif action == 3:  # Move Left
        next_col -= 1
    """
    station_size = 4
    total_reward = 0
    total_reward_shaped = 0
    cnt = [0,0,0]
    averaged = [0,0]
    batch_size = 100



    for epoch in tqdm(range(episodes+600)):
        if epoch >= episodes:
            epsilon = 0
        grid_size = np.random.randint(5,11)
        obstacle_size = np.random.randint(grid_size*grid_size)
        obs,_ = env.reset()
        averaged[0]+=env.grid_size
        averaged[1]+=len(env.obstacles)
        obs,ifpickup,p_loc = obs
        done = False
        state = get_state_obs(obs,action=None)
        steps=0
        action_l=[]
        success = False
        has_pickup=False
        trajectory = []
        while not done:
            """if state[-1]!=(0,0,0,0):
              print(state[-1])"""
            action = np.random.choice(action_size,p=softmax(policy_table[state]))
            lst_pickup = pickup
            obs,reward,done,_ = env.step(action)
            obs,ifpickup,p_loc = obs
            next_state = get_state_obs(obs,action)
            total_reward += reward
            if done and reward>0:
                cnt[2]+=1
            reward_shaping = 0
            pickup, p_len, goal_len, _,_ , _ = next_state
            if not pickup:
                reward_shaping -= 0.01 * (p_len**2)
            reward_shaping -= 0.001 * (goal_len**2)
            reward += reward_shaping
            total_reward_shaped += reward
            trajectory.append((state,action,reward))
            state = next_state


            if lst_pickup==False and pickup:
                cnt[0]+=1
            elif lst_pickup==True and pickup==False:
                cnt[1]+=1
            if pickup!=ifpickup:
                print(pickup,ifpickup)
            assert(pickup==ifpickup)
        G = 0
        for t in reversed(trajectory):
            state,action,reward = t
            G = gamma*G + reward
            prob = softmax(policy_table[state])
            grad=-prob.copy()
            grad[action]+=1
            policy_table[state] += alpha*G*grad
        if (epoch+1)%batch_size==0:
            cnt = [i/batch_size for i in cnt]
            print(f'Epoch : {epoch}, average reward : {total_reward/batch_size:.4f}, averaged shaped reward : {total_reward_shaped/batch_size:.4f} Pickup, Drop, Success rate : {cnt}')
            print(f'averaged grid size : {averaged[0]/batch_size:.2f}, averaged obstacles : {averaged[1]/batch_size:.2f}')
            averaged = [0,0]
            cnt = [0,0,0]
            total_reward = 0
            total_reward_shaped = 0
    return q_table

In [45]:
env = SimpleTaxiEnv()
policy_table = tabular_policy_learning(env)
# save policy table to .pkl
policy_dict = dict(policy_table)  # Convert to regular dict
with open('policy_table.pkl', 'wb') as f:
    pickle.dump(policy_dict, f)

  0%|          | 20/5600 [00:04<20:19,  4.58it/s]


KeyboardInterrupt: 